Lie, Samuel Miracle Kristanto - 0706012210011
Nathan Darrell - 0706012210034

In [47]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

print("\n\nNumber of GPUs Available: ",
      len(tf.config.experimental.list_physical_devices('GPU')))

2.16.2
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3183232131077695874
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 3109936634932313060
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]


Number of GPUs Available:  1


2025-05-25 18:29:55.822464: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-25 18:29:55.822479: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Read Dataset

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

import warnings
warnings.filterwarnings('ignore')

In [49]:
data_dir = '/Users/attraqtia/Documents/ComVis/Main'
width = 512 # Ini sudah size image dari datasetna
height = 512
batch_size = 16

In [50]:
train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="training",
    seed=123,
    image_size=(height, width),
    batch_size=batch_size
)

validation_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="validation",
    seed=123,
    image_size=(height, width),
    batch_size=batch_size
)

Found 82838 files belonging to 2 classes.
Using 57987 files for training.
Found 82838 files belonging to 2 classes.
Using 24851 files for validation.


In [51]:
class_names = train_ds.class_names
print(class_names)

['fake', 'real']


## Model Building --> MobileNetV3

In [52]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, DepthwiseConv2D, GlobalAveragePooling2D, Dense, Add
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Recall, Precision

# Define the custom MobileNetV3-like block
def inverted_residual_block(x, input_channels, output_channels, expansion_factor, strides):
    x_input = x  # Save for residual connection
    expanded_channels = int(input_channels * expansion_factor)

    # Expansion (1x1 conv)
    if expansion_factor != 1:
        x = Conv2D(expanded_channels, kernel_size=1, padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    # Depthwise convolution
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Projection (1x1 conv)
    x = Conv2D(output_channels, kernel_size=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    # Residual connection (if input and output shapes match and stride == 1)
    if strides == 1 and input_channels == output_channels:
        x = Add()([x_input, x])

    return x


# Define the model
def MobileNetV3_Model(input_shape=(512, 512, 3), num_classes=1):
    input = Input(shape=input_shape)
    x = Conv2D(16, kernel_size=3, strides=2, padding='same', use_bias=False)(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNetV3-style blocks
    x = inverted_residual_block(x, 16, 16, expansion_factor=1, strides=1)
    x = inverted_residual_block(x, 16, 24, expansion_factor=4, strides=2)
    x = inverted_residual_block(x, 24, 24, expansion_factor=3, strides=1)
    x = inverted_residual_block(x, 24, 40, expansion_factor=3, strides=2)
    x = inverted_residual_block(x, 40, 40, expansion_factor=3, strides=1)
    x = inverted_residual_block(x, 40, 80, expansion_factor=6, strides=2)
    x = inverted_residual_block(x, 80, 80, expansion_factor=2.5, strides=1)

    # Head
    x = Conv2D(1280, kernel_size=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='sigmoid')(x)

    model = Model(inputs=input, outputs=output)

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', Recall(), Precision()]
    )

    return model

In [53]:
model = MobileNetV3_Model(input_shape=(512, 512, 3))
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_47 (Conv2D)  │ (None, 256, 256,  │        432 │ input_layer_5[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ conv2d_47[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_46 (ReLU)     │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_22 │ (None, 256, 256,  │        144 │ re_lu_46[0][0]    │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_47 (ReLU)     │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_48 (Conv2D)  │ (None, 256, 256,  │        256 │ re_lu_47[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ conv2d_48[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_13 (Add)        │ (None, 256, 256,  │          0 │ re_lu_46[0][0],   │
│                     │ 16)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_49 (Conv2D)  │ (None, 256, 256,  │      1,024 │ add_13[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_49[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_48 (ReLU)     │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_23 │ (None, 128, 128,  │        576 │ re_lu_48[0][0]    │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_49 (ReLU)     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_50 (Conv2D)  │ (None, 128, 128,  │      1,536 │ re_lu_49[0][0]  

 Total params: 205,057 (801.00 KB)

 Trainable params: 198,753 (776.38 KB)

 Non-trainable params: 6,304 (24.62 KB)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

# blom donlot

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [ ]:
from datetime import datetime

start_time = datetime.now()

history = model.fit(
    train_ds,
    batch_size = batch_size,
    epochs = 5,
    validation_data = validation_ds,
    verbose = 1
)

end_time = datetime.now()

# myModel.save('/Users/attraqtia/Documents/ComVis/reyal_fakeh_model.h5')

Epoch 1/5


2025-05-25 18:29:59.272315: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 337/3625 ━━━━━━━━━━━━━━━━━━━━ 16:48 307ms/step - accuracy: 0.8373 - loss: 0.4433 - precision_2: 0.8448 - recall_2: 0.9887